# Uruchominie jointa w trybie torque

In [1]:
import can
import matplotlib as plt
import numpy as np
import os
import time
import math
from IPython.display import display, clear_output
from joint_can import *
%matplotlib inline


In [2]:
def initialize_joint(bus, mode: int, nr_id):
    send_msg(0x1A0 + nr_id, None, bus)
    time.sleep(1)
    
    send_msg(0x1b0 + nr_id, [0,0,0,0,0,0], bus)
    time.sleep(0.1)
    send_msg(0x140 + nr_id, None, bus)
    time.sleep(0.1)
    send_msg(0x1a0 + nr_id, None, bus)
    time.sleep(1)
    
    if (send_msg(0x1F0 + nr_id, [mode, 2], bus).data[0] == 1):
        send_msg(0x110 + nr_id, [2], bus)
        time.sleep(1)
        send_msg(0x110 + nr_id, [3], bus).data[0]
        time.sleep(5)
        return True
    else:
        return False

In [3]:
bus = can.interface.Bus(bustype='socketcan', channel='can0', bitrate=1000000, data_bitrate=5000000, fd=True)
nr_id = get_can_id(bus) #numer id jaki chcemy nadac do jointa
initialize_joint(bus,2,nr_id)

True

# Ustawienie torque'a

In [4]:
tq=16. #torque w Nm
duration = 300 #czas trwania w sekundach

# Blok główny programu

In [7]:
#blok pracuje w pętli, aby zatrzymać jointa trzeba zatrzymać ten blok i uruchomić kolejny

# fig = plt.pyplot.figure()
# ax = fig.add_subplot(1, 1, 1) 

spin=1
it=0
tq_data=[]

start=time.time()
elapsed=0
min_speed=2.
max_speed=0.
set_torque(tq,bus,nr_id)
time.sleep(5)

c_speed=0.
j_data=set_speed(c_speed,bus, nr_id)
data_t=get_temp(bus, nr_id)
#torque zwracany co 1s w Nm
# while(elapsed<duration):
#     elapsed = time.time()-start
#     data=set_torque(tq,bus)
#     tq_data.append(data.torque)
#     min_speed=min(min_speed,data.speed)
#     max_speed=max(max_speed,data.speed)
#     print('\r speed: {0:.3} (min: {3:.3} | max: {4:.3})  | time: {1}s / {2}s      '.format(data.speed, int(elapsed),duration, min_speed, max_speed), end='')
#     time.sleep(1)
    # ax.cla()
    # ax.plot(it, data.torque)
    # display(fig)
    # clear_output(wait = True)
    # it+=1
    # plt.pyplot.pause(5)
    
while(spin):
    j_data=set_torque(tq,bus, nr_id)
    data_t=get_temp(bus, nr_id)
    # elapsed = time.time()-start
    print(f'\r tq: {j_data.torque:.3f}\tpredkosc: {speed}\tm_temp: {data_t.motor_temp:.3f}\tb_temp: {data_t.bearing_temp:.3f}\tenkoder_abs: {j_data.position:.3f}', end='') #usunięta wartość: \tabs_inc_diff: {diff:.3f}
    time.sleep(0.02)
set_torque(0.,bus)

KeyboardInterrupt: 

# Zatrzymanie jointa

In [ ]:
set_torque(0.,bus)    #zatrzymanie ctrl-enter